In [1]:
import pandas as pd
import numpy as np
import KNN.similarities as sim
import KNN.utils as utils
from tqdm import tnrange, tqdm_notebook

from scipy.spatial.distance import pdist, squareform

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
users = list(set(pd.read_csv('data/experiment_data/ratings.csv').userId))
functions = ['binary', 'euclidean', 'cosine', 'manhattan']
k_list =[1, 2, 3, 5, 10]

In [3]:
matrix_binary = sim.get_binary_similarity_matrix()

In [4]:
def error_rating(user, algorithm, k, matrix_binary):
    # print(user)
    # Obtenemos los datos de las películas
    train_movies, test_movies = utils.get_train_test_movies(user)
    all_movies = utils.get_all_movies(user)
    
    # Calculamos la matriz filtrada de similitud
    if algorithm == 'binary':
        matrix = pd.read_csv('data/similarity_data/jaccard_matrix.csv')
    if algorithm == 'euclidean':
        matrix = pd.read_csv('data/similarity_data/sim_euclidean_user_{}.csv'.format(user))
    elif algorithm == 'cosine':
        matrix = pd.read_csv('data/similarity_data/sim_cosine_user_{}.csv'.format(user))
    elif algorithm == 'manhattan':
        matrix = pd.read_csv('data/similarity_data/sim_manhattan_user_{}.csv'.format(user))
        
    filtered_matrix = utils.filter_similarity_matrix(matrix, test_movies, train_movies, all_movies)
    return utils.calculate_mean_ratings(filtered_matrix, test_movies, user, k)
    

In [5]:
def average(errors_list):
    return sum(errors_list) / len(errors_list)

In [6]:
errorsDF = pd.DataFrame(columns=functions, index=k_list)

for f in tnrange(len(functions), desc='functions'):
    for k in tnrange(len(k_list), desc='K', leave=False):
        avg = []
        for u in tnrange(len(users), desc='users', leave=False):
            avg.append(error_rating(users[u], functions[f], k_list[k], matrix_binary))
        errorsDF[functions[f]][k_list[k]] = average(avg)

In [8]:
errorsDF.transpose()

,1,2,3,5,10
binary,0.732967,0.748771,0.754994,0.76001,0.76751
euclidean,0.782791,0.782697,0.784004,0.77789,0.77765
cosine,0.783354,0.786358,0.785175,0.782827,0.776097
manhattan,0.776746,0.786377,0.782373,0.778093,0.772596
